In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import os
import numpy as np
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 100
lr = 0.1
batch_size = 128
REGULARIZER  = 0.0001
checkpoint_save_path =  './Model/FractalNet/'

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.1,
               height_shift_range = 0.1,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.1,
               zoom_range = 0.1)
DataGenTrain.fit(x_train)

In [ ]:
def scheduler(epoch):  #HTD(-6,3)衰减
    start = -6.0
    end = 3.0
    return lr / 2.0 * (1- math.tanh( (end-start)*epoch/epochs + start))

In [ ]:
class ConvBNRelu(Model):
    def __init__(self,channels,kernel_size,strides,padding):
        super(ConvBNRelu,self).__init__()
        self.model = tf.keras.models.Sequential([
                    Conv2D(filters=channels, kernel_size=kernel_size,strides=strides, padding=padding,
                           kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER)),
                    BatchNormalization(momentum=0.9),
                    Activation('relu') ])
    def call(self,inputs):
        outputs = self.model(inputs)
        return outputs

In [ ]:
# 递归写法
# class FractalBlock(Model):
#     def BasicBlock(self,inputs,channels,columns):  
#         fz = ConvBNRelu(channels=channels,kernel_size=3,strides=1, padding='same')(inputs)
#         if column >= 1:
#             fc1 = self.BasicBlock(inputs, channels, column-1)
#             fc2 = self.BasicBlock(fc1, channels, column-1)
#             x = tf.concat([fz,fc2], axis = -1)
#             y = Activation("relu")(x)
#             return y
#         else:
#             return fz
        
#     def __init__(self,channels,columns):
#         super(FractalBlock,self).__init__()
#         self.columns = columns
#         self.block = self.BasicBlock(channels=channels,columns=columns)
        
#     def call(self,inputs):
#         outputs = self.block(inputs)
#         return outputs

In [ ]:
class FractalBlock(Model):
    def __init__(self,channels,columns):
        super(FractalBlock,self).__init__()
        self.convlist = []
        for _ in range(2**columns-1):
            self.convlist.append(ConvBNRelu(channels=channels,kernel_size=3,strides=1,padding='same'))
    def call(self,inputs):
        x = self.convlist[0](inputs)
        x = self.convlist[1](x)
        path = self.convlist[2](inputs)
        join = tf.keras.layers.average([x,path])
        path = self.convlist[3](join)
        x = self.convlist[4](join)
        x = self.convlist[5](x)
        join = tf.keras.layers.average([x,path])
        path = self.convlist[6](inputs)
        outputs = tf.keras.layers.average([join,path])
        return outputs

In [ ]:
class FractalNet(Model):
    def __init__(self,B,C):
        super(FractalNet,self).__init__()
        self.channels = [64,128,256,512,512]
        self.blocks = Sequential()
        for i in range(B):
            self.blocks.add(FractalBlock(channels=self.channels[i], columns=C))
            self.blocks.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))
        self.f1 = Dense(10,activation='softmax',kernel_initializer="he_normal",
                        kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    def call(self,inputs):
        x = self.blocks(inputs)
        y = self.f1(x)
        return y

In [ ]:
model = FractalNet(B=5,C=3)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True, clipnorm=2.),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

log_dir = os.path.join("Model","FractalNet_logs")
callbacks = [
            tf.keras.callbacks.LearningRateScheduler(scheduler),  #学习率衰减表
            #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=0.0001, patience=10, cooldown=0)
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=15, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))
# best result: 94.32%  (86epochs)

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/FractalNet_logs
#http://localhost:6006/